# 이미지 데이터 준비

In [1]:
import pymysql
prob_db = pymysql.connect(
    user='real', 
    passwd='vmfl515!dnlf', 
    host='mathflat-dev-cluster.cluster-ce1us4oyptfa.ap-northeast-2.rds.amazonaws.com', 
    db='iclass', 
    charset='utf8'
)

In [2]:
curs = prob_db.cursor(pymysql.cursors.DictCursor) #dataframe형태로 사용하기 위해 dictcursor사용

In [3]:
sql = 'SELECT * FROM iclass.Table_middle_problems WHERE unitCode LIKE "1%" ';
curs.execute(sql)

192267

In [4]:
result = curs.fetchall()

In [5]:
import pandas as pd

result = pd.DataFrame(result)
result.head()

,ID,unitCode,problemIDByUnit,problemLevel,problemType,problemURL,answerData,totalTimes,correctTimes,curriculumNumber,...,ClassID,NeedCheckLayout,DateTime_Add,DateTime_Update,Delimiter,PreviousConceptCode1,PreviousConceptCode2,PreviousConceptCode3,hwpExist,scorable
0,41649,14100003,0,1,객관식,/math_problems/9/e/4/1/0/9_141000_GbiCu_0_,2,38,34,9,...,0,0,NaT,2019-10-02 12:42:44,0,0,0,0,2,1
1,41650,14100003,1,1,주관식,/math_problems/9/e/4/1/0/9_141000_GbiCu_1_,None,61,55,9,...,0,0,NaT,2020-01-16 11:18:24,0,0,0,0,2,0
2,41651,14100003,2,1,객관식,/math_problems/9/e/4/1/0/9_141000_GbiCu_2_,5,59,53,9,...,0,0,NaT,2019-10-02 12:42:44,0,0,0,0,2,1
3,41652,14100003,3,2,주관식,/math_problems/9/e/4/1/0/9_141000_GbiCu_3_,None,62,55,9,...,0,0,NaT,2019-10-02 12:42:44,0,0,0,0,2,0
4,41653,14100003,4,1,주관식,/math_problems/9/e/4/1/0/9_141000_GbiCu_4_,None,53,47,9,...,0,0,NaT,2019-10-18 18:11:48,0,0,0,0,2,0


# 필요한 컬럼만 가져옴

In [6]:
result = result[['ID','unitCode','problemLevel','problemType','problemURL','curriculumNumber','NegativeFeedback']]

index를 ID로 바꿈

In [7]:
result.set_index("ID",inplace=True)

In [8]:
result.head()

,unitCode,problemLevel,problemType,problemURL,curriculumNumber,NegativeFeedback
ID,,,,,,
41649,14100003,1,객관식,/math_problems/9/e/4/1/0/9_141000_GbiCu_0_,9,5
41650,14100003,1,주관식,/math_problems/9/e/4/1/0/9_141000_GbiCu_1_,9,5
41651,14100003,1,객관식,/math_problems/9/e/4/1/0/9_141000_GbiCu_2_,9,5
41652,14100003,2,주관식,/math_problems/9/e/4/1/0/9_141000_GbiCu_3_,9,5
41653,14100003,1,주관식,/math_problems/9/e/4/1/0/9_141000_GbiCu_4_,9,5


# Dataframe에서 url를 가져옴

In [9]:
UrlList = result['problemURL']

In [10]:
import urllib.request
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [11]:
import requests

# 이미지 다운받지 않고 진행하기

<전체 과정>

    DataFrame에 저장된 url를 모은 UrlList에서 url를 이용하여 이미지를 불러옴 -> 파일 형식을 png에서 jpg로 바꿈 -> jpg파일를 torchvision transforms을 이용하여 rescaling -> Resnet18을 이용하여 feature vector로 바꿔서 추천

In [12]:
import torch
from tqdm import tqdm
from torchvision import models



class Img2VecResnet18():
    def __init__(self):
        
        if torch.cuda.is_available():
            self.device = torch.device("cuda")
        else:
            self.device = torch.device("cpu")
        self.numberFeatures = 512
        self.modelName = "resnet-18"
        self.model, self.featureLayer = self.getFeatureLayer()
        self.model = self.model.to(self.device)
        self.model.eval()
        self.toTensor = transforms.ToTensor()
        
        # normalize the resized images as expected by resnet18
        # [0.485, 0.456, 0.406] --> normalized mean value of ImageNet, [0.229, 0.224, 0.225] std of ImageNet
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        
    def getVec(self, img):
        image = self.normalize(self.toTensor(img)).unsqueeze(0).to(self.device)
        embedding = torch.zeros(1, self.numberFeatures, 1, 1)

        def copyData(m, i, o): embedding.copy_(o.data)

        h = self.featureLayer.register_forward_hook(copyData)
        self.model(image)
        h.remove()

        return embedding.numpy()[0, :, 0, 0]

    def getFeatureLayer(self):
        
        cnnModel = models.resnet18(pretrained=True)
        layer = cnnModel._modules.get('avgpool')
        self.layer_output_size = 512
        
        return cnnModel, layer
        

allVectors에 dictionary형태로 feature vector저장 ( key : url, value : feature vector )

In [ ]:
import os,io
from PIL import Image
from torchvision import transforms

# generate vectors for all the images in the set
img2vec = Img2VecResnet18() 

allVectors = {}
inputDim = (224,224)
print("Converting images to feature vectors:")

unavailable_url = []

for ind in tqdm(list(UrlList.index)):
    
    url = UrlList[ind]
    url = "https://s3.ap-northeast-2.amazonaws.com/mathflat"+ url + "p.png"
    url = url.replace("/math_problems/","/math_problems/d/")
   
    try:
        
        res = requests.get(url)
        img = Image.open(io.BytesIO(res.content)).convert('RGB') #png image to jpg
        
        # image crop -> 224X224 sized
        
        transformationForCNNInput = transforms.Compose([transforms.Resize(inputDim)])
        newI = transformationForCNNInput(img)
    
        # ResNet18 이용해서 feature vector로 변환
       
        vec = img2vec.getVec(newI)
        
        allVectors[ind] = vec
        
    except: 
        
        unavailable_url.append(url)
        pass
   

  0%|          | 2/192267 [00:00<3:51:25, 13.85it/s]

Converting images to feature vectors:


 57%|█████▋    | 109288/192267 [3:19:57<3:13:31,  7.15it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 70%|███████   | 134707/192267 [4:06:28<1:40:40,  9.53it/s] 

# Cosine similarity
Calculate for all vectors the cosine similarity to the other vectors.
Note that this matrix may become huge, hence infefficient, with many thousands of images

            같은 UnitCode인 vector랑만 비교하면 된다...문제 key를 url로 했는데 ID로 해야 같은 UnitCode문제를 불러오기 쉽지 않을까

In [15]:
import pickle
with open("FeatureVectors.txt",'wb') as f:
    pickle.dump(allVectors,f)

In [17]:
len(allVectors.keys())

250121

In [16]:
# now let us define a function that calculates the cosine similarity entries in the similarity matrix
import pandas as pd
import numpy as np

def getSimilarityMatrix(vectors):
    v = np.array(list(vectors.values())).T
    sim = np.inner(v.T, v.T) / ((np.linalg.norm(v, axis=0).reshape(-1,1)) * ((np.linalg.norm(v, axis=0).reshape(-1,1)).T))
    keys = list(vectors.keys())
    matrix = pd.DataFrame(sim, columns = keys, index = keys)
    
    return matrix
        
similarityMatrix = getSimilarityMatrix(allVectors)

MemoryError: Unable to allocate array with shape (250121, 250121) and data type float32

In [ ]:
allVectors.keys()

# Prepare top-k lists
Now that the similarity matrix is fully available, the last step is to sort the values per item and store the top similar entries in another data structure

In [ ]:
from numpy.testing import assert_almost_equal

k = 4 # the number of top similar images to be stored

similarNames = pd.DataFrame(index = similarityMatrix.index, columns = range(k))
similarValues = pd.DataFrame(index = similarityMatrix.index, columns = range(k))

for j in tqdm(range(similarityMatrix.shape[0])):
    kSimilar = similarityMatrix.iloc[j, :].sort_values(ascending = False).head(k)
    similarNames.iloc[j, :] = list(kSimilar.index)
    similarValues.iloc[j, :] = kSimilar.values

In [ ]:
print(similarNames)

# Get and visualize similar images for four example inputs

In [ ]:
import matplotlib.pyplot as plt
    
# take three examples from the provided image set and plot
inputImages = ["1.jpg", "test25.jpg", "test30.jpg", "7.jpg"]

numCol = 4
numRow = 1

def setAxes(ax, image, query = False, **kwargs):
    value = kwargs.get("value", None)
    if query:
        ax.set_xlabel("Query Image\n{0}".format(image), fontsize = 12)
    else:
        ax.set_xlabel("Similarity value {1:1.3f}\n{0}".format( image,  value), fontsize = 12)
    ax.set_xticks([])
    ax.set_yticks([])
    
def getSimilarImages(image, simNames, simVals):
    if image in set(simNames.index):
        imgs = list(simNames.loc[image, :])
        vals = list(simVals.loc[image, :])
        if image in imgs:
            assert_almost_equal(max(vals), 1, decimal = 5)
            imgs.remove(image)
            vals.remove(max(vals))
        return imgs, vals
    else:
        print("'{}' Unknown image".format(image))
        
def plotSimilarImages(image, simiarNames, similarValues):
    simImages, simValues = getSimilarImages(image, similarNames, similarValues)
    fig = plt.figure(figsize=(10, 20))
    
    # now plot the  most simliar images
    for j in range(0, numCol*numRow):
        ax = []
        if j == 0:
            img = Image.open(os.path.join(inputDir, image))
            ax = fig.add_subplot(numRow, numCol, 1)
            setAxes(ax, image, query = True)
        else:
            img = Image.open(os.path.join(inputDir, simImages[j-1]))
            ax.append(fig.add_subplot(numRow, numCol, j+1))
            setAxes(ax[-1], simImages[j-1], value = simValues[j-1])
        img = img.convert('RGB')
        plt.imshow(img)
        img.close()
    
    plt.show()
        
for image in inputImages:
    plotSimilarImages(image, similarNames, similarValues)

In [ ]:
op0